In [1]:
import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import data
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from models import ERC_MMN
from pickle_loader import load_erc

In [2]:
batch_size = 8
seq_len = 15
seq2_len = seq_len
emb_size = 768
hidden_size = 768
batch_first = True

In [27]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")

idx2utt, utt2idx, idx2emo, emo2idx, idx2speaker,\
    speaker2idx, weight_matrix, my_dataset_train, my_dataset_test,\
    final_speaker_info, final_speaker_dialogues, final_speaker_emotions,\
    final_speaker_indices, final_utt_len = load_erc()

weight_matrix = weight_matrix.to(device)
train_cnt = len(my_dataset_train)

In [31]:
print('Memory Usage:')
print('Allocated:', round(torch.cuda.memory_allocated(1)/1024**3,1), 'GB')
print('Cached:   ', round(torch.cuda.memory_reserved(1)/1024**3,1), 'GB')

Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [22]:
from torch.utils.data.dataset import random_split
# Define the sizes for the training and validation sets
total_size = len(my_dataset_train)
train_size = int(0.8 * total_size)
val_size = total_size - train_size

# Use random_split to split the dataset
train_dataset, val_dataset = random_split(my_dataset_train, [train_size, val_size])

In [23]:
print("train size: ", len(train_dataset))   
print("val size: ", len(val_dataset))

train size:  273
val size:  69


In [5]:
for d in my_dataset_train[0][1]:
    print(idx2utt[d.item()])

kya kya bhar ke rakha hai indravadan ne is ghar mein indravadan please tum sari bekar ki chijen bahar kyon nahin fekte
ok chalo rosesh chalo bahar
momma hath chhodiye dad
dekho maya ya na bol raha hai fir tum hi kahogi faltu chijen jama karte ho fekte nahin ho chalo
indravadan tum kitne salo se ye joke maar rahe ho koi hota hai aaj tak
kaikai pe joke maroge to manthra thodi na hasegi
very funny
maya monisha aur sahil aa rahe hain dekhna yahi jo un per istemal karta hun aur dekhna vah kitna haste hain
hi guys
hi
hi darling achcha indravadan please ya kachra bahar fenko
ok maya chalo rosesh tumhen bahar fenk au
mom diwali ka lunch hamare ghar pe
aur mein khana fresh banane wali hun
i rest my case indravadan hase donon huh achcha ab to please kachra bahar fenk do


In [6]:
for d in my_dataset_train[0][2]:
    print(idx2emo[d.item()])

   

disgust
contempt
anger
neutral
neutral
contempt
contempt
neutral
joy
joy
neutral
neutral
joy
joy
neutral


In [7]:
print(idx2speaker[final_speaker_info[0][0]])

maya


In [8]:
print(final_speaker_emotions)

{0: {0: 0, 1: 7, 2: 3, 3: 5, 4: 5, 5: 7, 6: 7, 7: 5, 8: 1, 9: 1, 10: 5, 11: 5, 12: 1, 13: 1, 14: 5, 15: 5, 16: 1, 17: 5, 18: 7, 19: 5, 20: 1, 21: 7}, 1: {0: 1, 1: 5, 2: 1, 3: 6, 4: 5, 5: 1, 6: 5, 7: 5, 8: 5, 9: 5, 10: 5, 11: 5, 12: 6, 13: 1, 14: 7, 15: 1, 16: 5, 17: 1, 18: 7, 19: 1, 20: 5, 21: 1, 22: 5, 23: 5, 24: 5, 25: 6, 26: 6, 27: 5, 28: 5, 29: 5, 30: 6, 31: 5, 32: 5, 33: 5, 34: 5, 35: 1, 36: 1, 37: 1}, 2: {0: 5, 1: 4, 2: 0, 3: 5, 4: 7, 5: 5, 6: 1, 7: 5, 8: 2, 9: 5, 10: 5, 11: 5, 12: 5, 13: 5, 14: 5, 15: 5, 16: 5, 17: 6, 18: 5, 19: 6, 20: 2, 21: 6, 22: 7, 23: 6, 24: 5, 25: 6, 26: 7, 27: 6, 28: 5}, 3: {0: 3, 1: 5, 2: 5, 3: 5, 4: 5, 5: 5, 6: 5, 7: 5, 8: 3, 9: 6, 10: 5, 11: 6, 12: 5, 13: 6, 14: 5, 15: 0, 16: 7, 17: 5, 18: 6, 19: 5, 20: 5, 21: 6, 22: 3, 23: 6, 24: 5, 25: 6, 26: 3, 27: 5, 28: 5, 29: 6, 30: 5, 31: 6, 32: 3}, 4: {0: 6, 1: 5, 2: 7, 3: 5, 4: 6, 5: 6, 6: 6, 7: 5, 8: 6, 9: 5, 10: 6, 11: 5, 12: 6, 13: 6, 14: 7, 15: 5, 16: 6, 17: 6, 18: 6, 19: 5, 20: 5, 21: 6, 22: 2}, 5: {0: 1,

In [6]:
def get_train_test_loader(bs):
    train_data_iter = data.DataLoader(my_dataset_train,batch_size=bs)
    test_data_iter = data.DataLoader(my_dataset_test,batch_size=bs,drop_last=True)
    
    return train_data_iter, test_data_iter

In [7]:
model = ERC_MMN(hidden_size,weight_matrix,utt2idx,1,seq_len).to(device)
weights1 = [1.0]*8
data_iter_train, data_iter_test = get_train_test_loader(1)


In [8]:
epochs = 100
class_weights1 = torch.FloatTensor(weights1).to(device)
criterion1 = nn.CrossEntropyLoss(weight=class_weights1,reduction='none').to(device)

optimizer = torch.optim.Adam(model.parameters(),lr=1e-3,weight_decay=1e-5)    
max_f1_1 = 0

In [9]:
model.train()

ERC_MMN(
  (ia): interact(
    (embedding): Embedding(7830, 768, padding_idx=272)
    (rnnD): myRNN(
      (RNN): GRU(768, 768, batch_first=True)
    )
    (drop1): Dropout(p=0.5, inplace=False)
    (rnnG): myRNN(
      (RNN): GRU(2304, 768, batch_first=True)
    )
    (drop2): Dropout(p=0.5, inplace=False)
    (attn): attention(
      (for_key): Linear(in_features=768, out_features=768, bias=True)
      (for_query): Linear(in_features=768, out_features=768, bias=True)
      (for_value): Linear(in_features=768, out_features=768, bias=True)
    )
    (rnnS): myRNN(
      (RNN): GRU(1536, 1536, batch_first=True)
    )
    (drop3): Dropout(p=0.5, inplace=False)
  )
  (mn): memnet(
    (rnn): myRNN(
      (RNN): GRU(768, 768, batch_first=True)
    )
    (masked_attention): maskedattn(
      (attn): attention(
        (for_key): Linear(in_features=768, out_features=768, bias=True)
        (for_query): Linear(in_features=1536, out_features=768, bias=True)
        (for_value): Linear(in_featu

In [10]:
avg_loss = 0
        
y_true1 = []
y_pred1 = []

In [11]:
i_batch,sample_batched = next(enumerate(data_iter_train))

0 has dialogue id 
1 has utterances 
2 has emotions

In [13]:
print(sample_batched[1])

tensor([[4530, 1588, 4270, 6376, 1238, 7074, 3653, 3863, 3937,   88, 4519, 5265,
         4602, 3963, 6278]])


In [35]:
dialogue_ids = sample_batched[0].tolist()            
inputs = sample_batched[1].to(device)
targets1 = sample_batched[2].to(device)

In [36]:
optimizer.zero_grad()

In [37]:
_, outputs = model(dialogue_ids, final_speaker_info, final_speaker_dialogues, final_speaker_emotions, final_speaker_indices, inputs)

In [40]:
print(outputs.size()[0])
print(final_utt_len[dialogue_ids[0]])

1
15


In [ ]:
s = 2
pred1 = torch.unsqueeze(outputs[0][s],dim=0).to(device)
truth1 = torch.LongTensor([targets1[0][s].item()]).to(device)


In [43]:
pred_emo = torch.argmax(F.softmax(pred1,-1),-1)

In [49]:
print(pred1[0][3])

tensor(-0., device='cuda:1', grad_fn=<SelectBackward0>)
